In [2]:
import skimage
import numpy as np
from skimage import io, transform
import os
import shutil
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import tensorflow as tf
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import urllib.request
import urllib.error
from moviepy.editor import VideoFileClip
import moviepy
import collections
from utils import label_map_util
from utils import visualization_utils as vis_util

import scipy as scp

%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Before opening the Jupyter Notebook make sure you have cloned the `models` folder into the repository root directory and run the following from the root diretory to install the TensorFlow API

```bash
git clone https://github.com/tensorflow/models.git
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
cd ..
cd ..
```

Set Up Path Directories
--------------------------

In [ ]:
root = os.getcwd()
imagePath = os.path.join(root, 'images')
trainPath = os.path.join(imagePath, 'train')
testPath = os.path.join(imagePath, 'test')

Convert XML Labels to CSV
-----------------------------

In [ ]:
# Modified From:
# https://github.comr/datitran/raccoon_dataset/blob/master/xml_to_csv.py

os.chdir(root)

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text+".png",
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[2][0].text),
                     int(member[2][1].text),
                     int(member[2][2].text),
                     int(member[2][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    
    for i in [trainPath, testPath]:
        image_path = i
        folder = os.path.basename(os.path.normpath(i))
        xml_df = xml_to_csv(image_path)
        xml_df.to_csv('data/'+folder+'.csv', index=None)
        print('Successfully converted xml to csv.')
    
main()

Create TF Record
------------------------------------------------------

When training models with TensorFlow using [tfrecords](http://goo.gl/oEyYyR) files help optimize your data feed.  We can generate a tfrecord using code adapted from this [raccoon detector](https://github.com/datitran/raccoon_dataset/blob/master/generate_tfrecord.py).

In [ ]:
%%bash 

python generate_tfrecord.py
mv test.record data
mv train.record data

Download Model
----------------

There are [models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) in the TensorFlow API that you can use depending on your needs.  If you want a high speed model that can work on detecting video feed at high fps the [single shot detection](http://www.cs.unc.edu/%7Ewliu/papers/ssd.pdf) model works best, but you gain speed at the cost of accuracy. Some object detection models detect objects by sliding different sized boxes across the image running the classifier many time on different sections of the image, this of course can be very resource consuming.  As it’s name suggests single shot detection determines all bounding box probabilities in one go, hence why it is a vastly faster model. I’ve already configured the [config](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs) file for mobilenet and included it in the GitHub repository for this post.  Depending on your computer you may have to lower the batch size in the config file if you run out of memory.



In [1]:
%%bash

#wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_11_06_2017.tar.gz
#tar xvzf ssd_mobilenet_v1_coco_11_06_2017.tar.gz

wget http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_2018_01_28.tar.gz
tar xvzf faster_rcnn_resnet101_coco_2018_01_28.tar.gz

Process is interrupted.


Train Model
-------------
Since we are only retraining the last layer of our pre-trained model a high end gpu is not required (but certainly can speed things up). Training time should roughly take an hour.  It will be much easier to watch the training process if you copy and paste the following code into a new terminal in the repository root directory.  Once our loss drops to a consistant level for a good while we can stop TensorFlow training by pressing ctrl+c.

To train the model copy and paste the following code into a new terminal from the repository root directory.  If using Docker create a new terminal pressing `ctrl` + `b` then `c`.

```bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
cd ..
cd ..

python3 models/research/object_detection/train.py --logtostderr --train_dir=data/ --pipeline_config_path=data/faster_rcnn_resnet50.config
```

Watch Training in TensorBoard
---------------------------------

We can use TensorBoard to monitor our total loss and other variables.  From the repository root directory run this command.

```bash
tensorboard --logdir='data'
```

Copy Object Detection Utilities to Our Root Directory
-------------------------------------------------------------

We just need to move some of the utilities from the Object Detection folder into our root path.

In [ ]:
%%bash

cp -R models/research/object_detection/utils/. utils

Export Inference Graph
-------------------------

I highly recommend you expiriment with different checkpoints as your model trains.  We can get a list of all the ckpt files with the following.

In [ ]:
%%bash 
cd data
ls model*.index

You can then added the cpkt number to our trained_checkpoint argument.

In [ ]:
%%bash 
rm -rf object_detection_graph
python models/research/object_detection/export_inference_graph.py --input_type image_tensor --pipeline_config_path data/ssd_mobilenet_v1_pets.config --trained_checkpoint_prefix data/model.ckpt-81956 --output_directory object_detection_graph

Test Model
-----------

In [4]:
# Modified From API
# https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = 'object_detection_graph/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'data/label_map.pbtxt'

NUM_CLASSES = 2

PATH_TO_TEST_IMAGES_DIR = 'images/validation/'
PATH_TO_SAVE_PROCESSED_IMAGES = 'images/validation_output/'
TEST_IMAGE_PATHS = os.listdir(PATH_TO_TEST_IMAGES_DIR)

IMAGE_SIZE = (12, 12)

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(
        np.uint8)


In [5]:
def run_inference_for_single_image(image, sess):
    # Get handles to input and output tensors
    ops = tf.get_default_graph().get_operations()
    all_tensor_names = {output.name for op in ops for output in op.outputs}
    tensor_dict = {}
    for key in ['num_detections', 'detection_boxes', 'detection_scores', 'detection_classes', 'detection_masks']:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
            tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
    if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.3), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
    image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

    # Run inference
    output_dict = sess.run(tensor_dict, feed_dict={image_tensor: np.expand_dims(image, 0)})

    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
      'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

#output_df = pd.DataFrame(output_dict)


In [ ]:
count = 0
intersections = {}
image_test = []
num_detections = []
detected_classes = []
detected_scores = []
ramp_dict = {}


with detection_graph.as_default():
    config = tf.ConfigProto()
    #config.gpu_options.per_process_gpu_memory_fraction = 0.7
    config.gpu_options.allow_growth=True

    with tf.Session(config=config) as sess:
        ops = detection_graph.get_operations()
        
        for image_path in TEST_IMAGE_PATHS:
            count += 1
            print("Processing image {}".format(count))
            image = Image.open(PATH_TO_TEST_IMAGES_DIR+image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            output_dict = run_inference_for_single_image(image_np, sess)
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              output_dict['detection_boxes'],
              output_dict['detection_classes'],
              output_dict['detection_scores'],
              category_index,
              instance_masks=output_dict.get('detection_masks'),
              use_normalized_coordinates=True,
              line_thickness=4)

            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)
            vis_util.save_image_array_as_png(image_np, PATH_TO_SAVE_PROCESSED_IMAGES+image_path)
            
            if output_dict['num_detections'] > 0:
                for i in range(output_dict['num_detections']):
                    image_test.append(image_path)
                    detected_classes.append(output_dict['detection_classes'][i])
                    detected_scores.append(output_dict['detection_scores'][i])
            else:
                image_test.append(image_path)
                detected_classes.append("NA")
                detected_scores.append("NA")

In [ ]:
df = pd.DataFrame()
df['image'] = image_test
df['image_id'] = df['image'].apply(lambda x: x.split("_")[0])
df['class_number'] = detected_classes
df['class'] = df['class_number'].apply(lambda x: "yellow" if x == 1 else ("gray" if x == 2 else "NA"))
df['score'] = detected_scores
df

In [ ]:
df2 =df.copy()
df2 = df2[df2["class"]=='NA']

In [ ]:
"""import shutil
origin_dir = "./images/validation_output/"
dest_dir = "./images/validation_failed/"

for index, row in df2.iterrows():
    shutil.copy2(origin_dir+row["image"], dest_dir)
"""

In [ ]:
"""with open("failed_2.csv", "w") as f:
    f.write(df2.to_csv())
"""

### Test on videos

In [8]:
videos = ["01"]
for t in videos:
    #del output_clip
    output_name = 'out_01_hq.mp4'
    input_video = 'videos/clip_000001.mov'
    with detection_graph.as_default():
        config = tf.ConfigProto()
        config.gpu_options.per_process_gpu_memory_fraction = 0.7
        #config.gpu_options.allow_growth=True
        with tf.Session(graph=detection_graph, config=config) as sess:
            def pipeline(image):
                image_np = image
                # Definite input and output Tensors for detection_graph
                image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                # Each box represents a part of the image where a particular object was detected.
                detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                # Each score represent how level of confidence for each of the objects.
                # Score is shown on the result image, together with the class label.
                detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
                detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
                num_detections = detection_graph.get_tensor_by_name('num_detections:0')
                #image = Image.open(image)
                # the array based representation of the image will be used later in order to prepare the
                # result image with boxes and labels on it.
                #image_np = load_image_into_numpy_array(image)
                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                image_np_expanded = np.expand_dims(image_np, axis=0)
                # Actual detection.
                (boxes, scores, classes, num) = sess.run(
                  [detection_boxes, detection_scores, detection_classes, num_detections],
                  feed_dict={image_tensor: image_np_expanded})
                # Visualization of the results of a detection.
                #(boxes, scores, classes, num) = reduce_boxes(boxes, scores, classes, num)
                visualize_boxes_and_labels_on_image_array(
                  image_np,
                  np.squeeze(boxes),
                  np.squeeze(classes).astype(np.int32),
                  np.squeeze(scores),
                  category_index,
                  use_normalized_coordinates=True,
                  line_thickness=2)
                # save Image
                return image_np
            
            clip = VideoFileClip(input_video)
            clip = moviepy.video.fx.all.rotate(clip, -90, unit='deg', resample='bicubic', expand=True)
            #clip = clip.add_mask().rotate(90)
            output_clip = clip.fl_image(pipeline)
            output_clip.write_videofile(output_name, audio=False, fps=60)
            #output_clip.write_videofile(output_name, audio=False)

[MoviePy] >>>> Building video out_01_hq.mp4
[MoviePy] Writing video out_01_hq.mp4


100%|██████████████████████████████████████████████████████████████████████████████| 1207/1207 [04:16<00:00,  5.30it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: out_01_hq.mp4 

